# Deep Graph Library (DGL) test

## Import libraries

In [35]:
import os
import sys
import shutil

os.environ["DGLBACKEND"] = "pytorch"
os.environ["WANDB_NOTEBOOK_NAME"] = "dgl.ipynb" #CHANGEEEEEEEEEEEEEEE

import dgl
import dgl.nn as dglnn
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from dgl.data import DGLDataset
from dgl import save_graphs, load_graphs
from dgl.data.utils import makedirs, save_info, load_info, split_dataset
from dgl.dataloading import GraphDataLoader
from torchmetrics.regression import MeanAbsolutePercentageError
from torchmetrics.regression import SymmetricMeanAbsolutePercentageError
from dgl import RowFeatNormalizer

torch.set_default_tensor_type(torch.DoubleTensor)

from datetime import datetime
import networkx as nx
import numpy as np
from datanetAPI import DatanetAPI
import requests
import configparser
import matplotlib.pyplot as plt
from tqdm import tqdm
from ast import literal_eval
from numpy import log as ln
import random

In [62]:
dataset_path = '/data/escenari-slicing/samples-UPC/V2-full'
save_dataset_path = './V2-samples-fulldataset'
save_model_path = './V2-model-first-long-train-fulldataset-multipleprediction-leaky'
dataset_name="Slicing-V2-fulldataset-865"

## Weights and Biases + Telegram config

In [3]:
# 1. Start a W&B run
# wandb.init(project="DGL-Slicing", entity="mfarreras",     
#            # track hyperparameters and run metadata
#             config={
#                 "learning_rate": 0.001,
#                 "model": "DESCONEGUT",
#                 "dataset": "TRAIN ANTIC",
#                 "epochs": 10,
#             })

#wandb.run.name = "First model"

In [4]:
# wandb_config.learning_rate = float(config['HYPERPARAMETERS']['learning_rate'])
# wandb_config.logs = config['DIRECTORIES']['logs']
# wandb_config.val = config['DIRECTORIES']['val']
# wandb_config.train = config['DIRECTORIES']['train']
# wandb_config.link_state_dim = config['HYPERPARAMETERS']['link_state_dim']
# wandb_config.path_state_dim = config['HYPERPARAMETERS']['path_state_dim']
# wandb_config.t = config['HYPERPARAMETERS']['t']
# wandb_config.readout_units = config['HYPERPARAMETERS']['readout_units']
# wandb_config.steps_per_epoch = config['RUN_CONFIG']['steps_per_epoch']
# wandb_config.validation_steps = config['RUN_CONFIG']['validation_steps']

## Dataset reading

In [65]:
class SlicingDataset(DGLDataset):
    """ 
    Parameters
    ----------
    raw_dir : str
        Specifying the directory that will store the
        downloaded data or the directory that
        already stores the input data.
    save_dir : str
        Directory to save the processed dataset.
        Default: the value of `raw_dir`
    force_reload : bool
        Whether to reload the dataset. Default: False
    verbose : bool
        Whether to print out progress information
    """
        
    def __init__(self, 
                 name="default_name", 
                 raw_dir=None, 
                 save_dir=None, 
                 force_reload=False):
        self.tool = DatanetAPI(raw_dir, shuffle=False)
        super(SlicingDataset, self).__init__(
            name=name, 
            raw_dir=raw_dir, 
            save_dir=save_dir, 
            force_reload=force_reload)
        

    def process(self):
        it = iter(self.tool)
        self.graphs = []
        for s in tqdm(it, total=len(self.tool.get_available_files()*7)):
            #obtain edge and graph data
            #how to save path data?
            G = nx.DiGraph(s.get_topology_object().copy())
            R = s.get_routing_matrix()
            T = s.get_traffic_matrix() 
            D = s.get_performance_matrix()
            P = s.get_port_stats()
            S = s.get_slices()
            D_G = nx.DiGraph()

            #AvgBw_sum calculation
            AvgBw_sum = np.zeros((len(G.nodes),len(G.nodes)))
            for src in range(G.number_of_nodes()):
                for dst in range(G.number_of_nodes()):
                    route = R[src,dst]
                    if route:
                        for index, node in enumerate(route):
                                next_node_index = index + 1
                                if next_node_index < len(route):
                                    for f_id in range(len(T[src, dst]['Flows'])):
                                        if T[src, dst]['Flows'][f_id]['AvgBw'] != 0 and T[src, dst]['Flows'][f_id]['PktsGen'] != 0:
                                            AvgBw_sum[node][route[next_node_index]] += T[src, dst]['Flows'][f_id]['AvgBw']

            slice_types = np.zeros((len(G.nodes),len(G.nodes)))
            reservations_embb = np.zeros((len(G.nodes),len(G.nodes)))
            reservations_urllc = np.zeros((len(G.nodes),len(G.nodes)))
            reservations_mmtc = np.zeros((len(G.nodes),len(G.nodes)))
            for slice in S:
                for flow in slice["flows"]:
                    route = literal_eval(flow["path"])
                    if slice["type"] == "eMBB":
                        sliceType = 0
                    elif slice["type"] == "URLLC":
                        sliceType = 1
                    else :
                        sliceType = 2
                    slice_types[route[0]][route[len(route)-1]] = sliceType
                    for index, node in enumerate(route):
                        next_node_index = index + 1
                        if next_node_index < len(route):
                            if (slice["type"] == "eMBB"):
                                v_max = -float(flow["bandwidth"])/ln(0.50)
                                v_min = -float(flow["bandwidth"])/ln(0.05)
                                reservations_embb[node][route[next_node_index]] += 0.5*v_min+slice["delta"]*(1.5*v_max-0.5*v_min)
                            elif (slice["type"] == "URLLC"):
                                reservations_urllc[node][route[next_node_index]] += flow["bandwidth"]
                            else:
                                reservations_mmtc[node][route[next_node_index]] += flow["bandwidth"]
            
            for src in range(G.number_of_nodes()):
                for dst in range(G.number_of_nodes()):
                    if src != dst:
        
                        if G.has_edge(src, dst):
                            D_G.add_node('l_{}_{}'.format(src, dst),
                                        capacity=torch.tensor(G.edges[src, dst]['bandwidth']),
                                        utilization=torch.tensor(P[src][dst]["utilization"]),
                                        losses=torch.tensor(P[src][dst]["losses"]),
                                        avgPacketSize=torch.tensor(P[src][dst]["avgPacketSize"]), 
                                        offeredTrafficIntensity=torch.tensor(AvgBw_sum[src][dst]/s.get_srcdst_link_bandwidth(src,dst)),#offeredTrafficIntensity
                                        embbReservations=torch.tensor(reservations_embb[src][dst]), #convertir a tant per 1 potser no ens va bé i millor a cues
                                        urllcReservations=torch.tensor(reservations_urllc[src][dst]),
                                        mmtcReservations=torch.tensor(reservations_mmtc[src][dst]),
                                        nodeType=torch.tensor(0)
                                        )
                            # for queue in queues:
                            #     new node with queue attributes
                            #     add edges from queues to links and from queues to flows
                            #     D_G.add_edge('p_{}_{}_{}'.format(src, dst, f_id), 'l_{}_{}'.format(h_1, h_2))
                            #     D_G.add_edge('l_{}_{}'.format(h_1, h_2), 'p_{}_{}_{}'.format(src, dst, f_id))
            for src in range(G.number_of_nodes()):
                for dst in range(G.number_of_nodes()):
                    if src != dst:
                        for f_id in range(len(T[src, dst]['Flows'])):
                            if T[src, dst]['Flows'][f_id]['AvgBw'] != 0 and T[src, dst]['Flows'][f_id]['PktsGen'] != 0:
                                D_G.add_node('p_{}_{}_{}'.format(src, dst, f_id),
                                            traffic=torch.tensor(T[src, dst]['Flows'][f_id]['AvgBw']),
                                            packets=torch.tensor(T[src, dst]['Flows'][f_id]['PktsGen']),
                                            delay=torch.tensor(D[src, dst]['Flows'][f_id]['AvgDelay']),
                                            jitter=torch.tensor(float(D[src, dst]['Flows'][f_id]['Jitter'])),
                                            pathLength=torch.tensor(len(R[src,dst])),
                                            drops=torch.tensor(D[src, dst]['AggInfo']['PktsDrop']/T[src, dst]['Flows'][f_id]['PktsGen']),
                                            delta=torch.tensor(slice["delta"]),#afegir reserva
                                            sliceType=torch.tensor(slice_types[src][dst]),
                                            nodeType=torch.tensor(1)
                                            )
        
                                for h_1, h_2 in [R[src, dst][i:i + 2] for i in range(0, len(R[src, dst]) - 1)]:
                                    D_G.add_edge('p_{}_{}_{}'.format(src, dst, f_id), 'l_{}_{}'.format(h_1, h_2), edgeType=torch.tensor(0))
                                    D_G.add_edge('l_{}_{}'.format(h_1, h_2), 'p_{}_{}_{}'.format(src, dst, f_id), edgeType=torch.tensor(1))

                        #afegir queues i slices vinculats a flows
                        #occupancy=P[src][dst]['qosQueuesStats'][0]['avgPortOccupancy']/G.nodes[src]['queueSizes'],
                        #drops=float(D[src, dst]['AggInfo']['PktsDrop'])/float(T[src, dst]['Flows'][0]['PktsGen']))
        
            D_G.remove_nodes_from([node for node, out_degree in D_G.out_degree() if out_degree == 0])
            
            g = dgl.from_networkx(D_G, node_attrs=["nodeType"])

            nodeTypes = nx.get_node_attributes(D_G, 'nodeType')
            nodes = D_G.nodes
            g.ndata[dgl.NTYPE] = torch.stack([nodeTypes[e] for e in nodes])
            g.ndata[dgl.NID] = torch.tensor([*range(0,len(D_G.nodes))])

            edgeTypes = nx.get_edge_attributes(D_G, 'edgeType')
            edges = D_G.edges
            g.edata[dgl.ETYPE] = torch.stack([edgeTypes[e] for e in edges])
            g.edata[dgl.EID] = torch.tensor([*range(0,len(D_G.edges))])
            g = dgl.to_heterogeneous(g, ['link','path'], ['traverses','composes'])

            capacities, utilizations, losses, avgPacketSizes, offeredTrafficIntensities, embbReservations, urllcReservations, mmtcReservations, nodeTypesLink = ([] for i in range(9))
            
            traffics, packets, delays, jitters, pathLengths, drops, deltas, nodeTypesPath, sliceTypesPath = ([] for i in range(9))
            
            for node in D_G.nodes(data=True):
                if node[1]["nodeType"] == 0: #LINK               
                    capacities.append(node[1]["capacity"])
                    utilizations.append(node[1]["utilization"])
                    losses.append(node[1]["losses"])
                    avgPacketSizes.append(node[1]["avgPacketSize"])
                    offeredTrafficIntensities.append(node[1]["offeredTrafficIntensity"])
                    embbReservations.append(node[1]["embbReservations"])
                    urllcReservations.append(node[1]["urllcReservations"])
                    mmtcReservations.append(node[1]["mmtcReservations"])
                    nodeTypesLink.append(node[1]["nodeType"])
            
                else :#if node[1]["type"] == 1: #FLOW/PATH
                    traffics.append(node[1]["traffic"])
                    packets.append(node[1]["packets"])
                    delays.append(node[1]["delay"])
                    jitters.append(node[1]["jitter"])
                    pathLengths.append(node[1]["pathLength"])
                    drops.append(node[1]["drops"])
                    deltas.append(node[1]["delta"])
                    nodeTypesPath.append(node[1]["nodeType"])
                    sliceTypesPath.append(node[1]["sliceType"])

            g.nodes["link"].data["capacity"] = torch.stack(capacities)
            g.nodes["link"].data["utilization"] = torch.stack(utilizations)
            g.nodes["link"].data["losses"] = torch.stack(losses)
            g.nodes["link"].data["avgPacketSize"] = torch.stack(avgPacketSizes)
            g.nodes["link"].data["offeredTrafficIntensity"] = torch.stack(offeredTrafficIntensities)
            g.nodes["link"].data["embbReservations"] = torch.stack(embbReservations)
            g.nodes["link"].data["urllcReservations"] = torch.stack(urllcReservations)
            g.nodes["link"].data["mmtcReservations"] = torch.stack(mmtcReservations)
            g.nodes["link"].data["nodeType"] = torch.stack(nodeTypesLink)
            
            g.nodes["path"].data["traffic"] = torch.stack(traffics)
            g.nodes["path"].data["packets"] = torch.stack(packets)
            g.nodes["path"].data["delay"] = torch.stack(delays)
            g.nodes["path"].data["jitter"] = torch.stack(jitters)
            g.nodes["path"].data["pathLength"] = torch.stack(pathLengths)
            g.nodes["path"].data["drops"] = torch.stack(drops)
            g.nodes["path"].data["delta"] = torch.stack(deltas)
            g.nodes["path"].data["nodeType"] = torch.stack(nodeTypesPath)
            g.nodes["path"].data["sliceType"] = torch.stack(sliceTypesPath)

            self.graphs.append(g)
            if len(self.graphs) > 865:
                break
        self.save()
        
    def getGraphs(self):
        return self.graphs

    
    def __getitem__(self, idx):
        """ Get graph and label by index
        Parameters
        ----------
        idx : int
            Item index

        Returns
        -------
        (dgl.DGLGraph)
        """
        return self.graphs[idx]

    def __len__(self):
        """Number of graphs in the dataset, *7 because the samples are grouped in groups of 7"""
        return len(self.tool.get_available_files())*7-1

    def collate_fn(self, batch):
        # batch is a list of tuple (graph, label)
        graphs = [e[0] for e in batch]
        g = dgl.batch(graphs)
        labels = [e[1] for e in batch]
        labels = torch.stack(labels, 0)
        return g

    def save(self):
        # save graphs and labels
        graph_path = os.path.join(self.save_path, 'dgl_graph.bin')
        save_graphs(graph_path, self.graphs)
        
    def load(self):
        # load processed data from directory `self.save_path`
        graph_path = os.path.join(self.save_path, 'dgl_graph.bin')
        self.graphs = load_graphs(graph_path)


    def has_cache(self):
        # check whether there is processed data in `self.save_path`
        graph_path = os.path.join(self.save_path, 'dgl_graph.bin')
        return os.path.exists(graph_path)

1. Obtenir graph base dirigit de cada sample  
2. Afegir atributs desitjats al graf
3. Carregar-lo a GPU

## Dataloader initialization

In [ ]:
# define dataset
dataset = SlicingDataset(
    name=dataset_name,
    raw_dir=dataset_path, 
    save_dir=save_dataset_path, 
    force_reload=False)

dataset.load()
ds_train, ds_validation, ds_test = split_dataset(dataset,[0.8, 0.1, 0.1]) #JUST FOR TEST
#ds_train, ds_validation, ds_test = split_dataset(dataset,[0, 0, 1]) #JUST FOR TEST

In [7]:
#set dataloader
#CANVIAR BATCH SIZE
#dataloader_train = GraphDataLoader(ds_train, batch_size=1, shuffle=False, collate_fn=dataset.collate_fn)
#dataloader_validation = GraphDataLoader(ds_validation, batch_size=1, shuffle=False, collate_fn=dataset.collate_fn)
#dataloader_test = GraphDataLoader(ds_test, batch_size=1, shuffle=False, collate_fn=dataset.collate_fn)

In [ ]:
print(dataset)

In [68]:
print(len(ds_train))
print(len(ds_validation))
print(len(ds_test))

0
0
8637


## GNN model

In [69]:
# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, in_feats_link, in_feats_path, hid_feats, out_feats_link, out_feats_path, rel_names):
        super().__init__()
        
        self.conv1 = dglnn.HeteroGraphConv({
            'traverses': dglnn.GraphConv(in_feats_path, hid_feats),
            'composes': dglnn.GraphConv(in_feats_link, hid_feats)},
            aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            'traverses': dglnn.GraphConv(hid_feats, out_feats_path),
            'composes': dglnn.GraphConv(hid_feats, out_feats_link)},
            aggregate='sum')
  
    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.leaky_relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [70]:
def joinFeatures(fs):
    features = []
    for feature in fs:
        if feature == "_ID" or feature == "nodeType" or feature == "utilization" or feature == "sliceType":
            continue
        else:
            features.append(fs[feature])
    return features

In [71]:
model = RGCN(7,7, 20, 3,3, ['traverses','composes'])
model.to(torch.double)

RGCN(
  (conv1): HeteroGraphConv(
    (mods): ModuleDict(
      (traverses): GraphConv(in=7, out=20, normalization=both, activation=None)
      (composes): GraphConv(in=7, out=20, normalization=both, activation=None)
    )
  )
  (conv2): HeteroGraphConv(
    (mods): ModuleDict(
      (traverses): GraphConv(in=20, out=3, normalization=both, activation=None)
      (composes): GraphConv(in=20, out=3, normalization=both, activation=None)
    )
  )
)

<h2> Training</h2>

In [72]:
#normalization
node_feats = ["avgPacketSize","embbReservations","urllcReservations","mmtcReservations","traffic","packets","delay","jitter","drops","delta"]

transform = RowFeatNormalizer(subtract_min=True, node_feat_names=node_feats)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_object = nn.MSELoss()

metric = MeanAbsolutePercentageError()
loss_values = []
val_values_delay = []
val_values_drops = []
val_values_jitter = []
batch_size = 100
i = 0
for epoch in tqdm(range(2000)): #wandb.config["epochs"] #5000+2000
    for g in random.sample(ds_train[0], batch_size):
        g = transform(g)
        
        link_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['link'].data)),0,1).contiguous()
        path_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['path'].data)),0,1).contiguous()
        labels = torch.stack((g.nodes['path'].data['delay'], g.nodes['path'].data['jitter'], g.nodes['path'].data['drops']))

        node_features = {'link': link_feats, 'path': path_feats}    
        model.train()
        # forward propagation by using all nodes and extracting the path embeddings
        logits = model(g, node_features)['path']
        logits = torch.transpose(logits,0,1)
        # compute loss
        loss = loss_object(logits, labels)
        loss_values.append(loss.item())
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        i+=1
    #break
    # validate model
    with torch.no_grad():
        for g in random.sample(ds_validation.dataset[0], batch_size):
            g = transform(g)
            link_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['link'].data)),0,1)
            path_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['path'].data)),0,1)
            labels = torch.stack((g.nodes['path'].data['delay'], g.nodes['path'].data['jitter'], g.nodes['path'].data['drops']))
            node_features = {'link': link_feats, 'path': path_feats}
            #out_data = torch.index_select(model(g,node_features)['path'],1,torch.tensor([2])).squeeze(1)
            logits = model(g, node_features)['path']
            logits = torch.transpose(logits,0,1)
            mape_delay = metric(logits[0], labels[0])
            val_values_delay.append(mape_delay.item())
            mape_jitter = metric(logits[1], labels[1])
            val_values_jitter.append(mape_jitter.item())
            mape_drops = metric(logits[2], labels[2])
            val_values_drops.append(mape_drops.item())
            
    # Save model
    torch.save(model.state_dict(), save_model_path)
    if epoch%100 == 0:
        print(mape_delay)
        print(mape_jitter)
        print(mape_drops)
    #wandb.log({"loss": loss, "MAPE_delay": mape, "MAPE_jitter": mape_jitter, "MAPE_drops": mape_drops})
#wandb.finish()

## Load the pre-trained model

In [73]:
model.load_state_dict(torch.load(save_model_path))
model.eval()

RGCN(
  (conv1): HeteroGraphConv(
    (mods): ModuleDict(
      (traverses): GraphConv(in=7, out=20, normalization=both, activation=None)
      (composes): GraphConv(in=7, out=20, normalization=both, activation=None)
    )
  )
  (conv2): HeteroGraphConv(
    (mods): ModuleDict(
      (traverses): GraphConv(in=20, out=3, normalization=both, activation=None)
      (composes): GraphConv(in=20, out=3, normalization=both, activation=None)
    )
  )
)

## Test prediction

In [74]:
metric = MeanAbsolutePercentageError()

In [95]:
import time
test_values_delay = []
test_values_drops = []
test_values_jitter = []

def find_indices(lst, element):
    indices = [index for index, value in enumerate(lst) if value == element]
    return indices

with torch.no_grad():
    start = time.time()
    for g in ds_test.dataset[0]:
        g = transform(g)
        link_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['link'].data)),0,1)
        path_feats = torch.transpose(torch.stack(joinFeatures(g.nodes['path'].data)),0,1)
        labels = torch.stack((g.nodes['path'].data['delay'], g.nodes['path'].data['jitter'], g.nodes['path'].data['drops']))
        node_features = {'link': link_feats, 'path': path_feats}
        
        logits = model(g, node_features)['path']
        logits = torch.transpose(logits,0,1)
        logits[0] = torch.nn.functional.relu(logits[0].clone(), inplace=True)
        logits[1] = torch.nn.functional.relu(logits[1].clone(), inplace=True)
        logits[2] = torch.nn.functional.relu(logits[2].clone(), inplace=True)
        sliceTypes = g.nodes['path'].data["sliceType"].tolist()
        #print(sliceTypes)
        indices = find_indices(sliceTypes, 1)
        if len(indices) == 0:
            continue
        else:
            #print(indices)
            logits_0 = torch.index_select(logits[0], 0, torch.tensor(indices))
            labels_0 = torch.index_select(labels[0], 0, torch.tensor(indices))
            logits_1 = torch.index_select(logits[1], 0, torch.tensor(indices))
            labels_1 = torch.index_select(labels[1], 0, torch.tensor(indices))
            logits_2 = torch.index_select(logits[2], 0, torch.tensor(indices))
            labels_2 = torch.index_select(labels[2], 0, torch.tensor(indices))
            
            mape_delay = metric(logits_0, labels_0)
            test_values_delay.append(mape_delay.item())
            mape_jitter = metric(logits_1, labels_1)
            test_values_jitter.append(mape_jitter.item())
            mape_drops = metric(logits_2, labels_2)
            test_values_drops.append(mape_drops.item())
    end = time.time()
    print(end - start)
        
print("MAPE delay: "+str(sum(test_values_delay)/len(test_values_delay)))
print("MAPE jitter: "+str(sum(test_values_jitter)/len(test_values_jitter)))
print("MAPE drops: "+str(sum(test_values_drops)/len(test_values_drops)))

2.4466090202331543
MAPE delay: 1.6823573528204716
MAPE jitter: 0.794666419700065
MAPE drops: 1.977382451851295
